In [25]:
import pandas as pd
import hashlib
from collections import defaultdict
from tqdm.auto import tqdm
import json

### Load Data

In [5]:
diabetes_raw_df = pd.read_parquet("hf://datasets/abdelhakimDZ/diabetes_QA_dataset/data/train-00000-of-00001.parquet")
diabetes_raw_df.head()

,question,answer
0,"What are some examples of calorie-free, caffei...","Examples of calorie-free, caffeine-free liquid..."
1,Why is it important for individuals with diabe...,Staying hydrated is crucial for regulating blo...
2,What type of gelatin is recommended for indivi...,Individuals with diabetes should choose gelati...
3,In what situations should an individual with d...,Individuals with diabetes should contact their...
4,Why is it important for individuals with diabe...,Wearing medical identification alerts others t...


In [7]:
diabetes_raw_df.drop_duplicates(inplace=True)

In [8]:
diabetes_raw_df.describe()

,question,answer
count,1071,1071
unique,1054,1065
top,"What are blood lipids, and why are they monito...",The booklet suggests incorporating activities ...
freq,3,2


* We have some duplicate values in Our data, Since they are few we nith have to drop them

### Adding index to our data

In [9]:
# Convert Dataframe to dictionary
diabetes_raw_dict = diabetes_raw_df.to_dict(orient='records')

In [16]:

def generate_document_id(doc):
    # combined = f"{doc['course']}-{doc['question']}"
    combined = f"{doc['question']}-{doc['answer'][:15]}"
    hash_object = hashlib.md5(combined.encode())
    hash_hex = hash_object.hexdigest()
    document_id = hash_hex[:8]
    return document_id

In [17]:
for doc in tqdm(diabetes_raw_dict):
    doc['id'] = generate_document_id(doc)

100%|████████████████████████████████████████████████████████████████████████████████| 1071/1071 [00:00<00:00, 539850.93it/s]


In [20]:
hashes = defaultdict(list)
for doc in diabetes_raw_dict:
    doc_id = doc['id']
    hashes[doc_id].append(doc)

In [26]:
## Save the data with ID

with open('diabetes_data_with_ids', 'wt') as f_out:
    json.dump(diabetes_raw_dict, f_out, indent=2)